Notebook to debug the MCS + NLLS approach.

In [ ]:
%load_ext autoreload
%autoreload 2
#%matplotlib qt

In [ ]:
import os
import sys
from pathlib import Path

In [ ]:
src_dir = str(Path(os.getcwd()).parent / "src")
sys.path.insert(1, src_dir)
#os.environ["PYTHONPATH"] = src_dir

In [ ]:
import data_loader
import calculate_concentr_diffs
from concentr_diffs_pathlength import *
from plotting import *
from mbll_functions import *
from utils import *
from mcs_function import *
from pathlib import Path
import config
import pickle
from ray import tune
import pmcx
import scipy

import matplotlib.pyplot as plt

In [ ]:
config.data_dir

In [ ]:
config.spectra_dir

In [ ]:
config.gpuid

In [ ]:
loader = data_loader.DataLoaderHELICOID(
    config.dataset_dir,
    wavelength_left_cut=520,
    wavelength_right_cut=900,
    #num_wavelengths=20
)

delta_A_gt_img = loader.get_attenuation_change("020-01")
A_gt_img = loader.get_attenuation("020-01")

mu_a_matrix_old = loader.absorption_coefs_old(
    use_diff_oxycco=False,
    use_water_and_fat=True
)

mu_a_matrix = loader.absorption_coefs(
    use_diff_oxycco=False,
    use_water_and_fat=True
)

mu_a_matrix_static = data_loader.DataLoader.absorption_coefs(
    loader.wavelengths,
    use_diff_oxycco=False,
    use_water_and_fat=True
)

print(np.allclose(mu_a_matrix, mu_a_matrix_static))
print(np.max(np.abs(mu_a_matrix - mu_a_matrix_static)))

In [ ]:
plot_spectra(mu_a_matrix_old[:, :4].T, loader.wavelengths, ["HbO2", "Hbb", "oxCCO", "redCCO", "Water", "Fat"])

In [ ]:
plot_spectra(mu_a_matrix[-500:-300, :].T, loader.wavelengths[-500:-300], ["HbO2", "Hbb", "oxCCO", "redCCO", "Water", "Fat"])

In [ ]:
plot_spectra(mu_a_matrix[-400:-500, :].T, loader.wavelengths[-400:-500], ["HbO2", "Hbb", "oxCCO", "redCCO", "Water", "Fat"])

In [ ]:
plot_spectra(mu_a_matrix[-300:, -2:].T, loader.wavelengths[-300:], ["Water", "Fat"])

In [ ]:
plot_spectra(mu_a_matrix_old[-300:, -2:].T, loader.wavelengths[-300:], ["Water", "Fat"])

In [ ]:
fat_data = np.loadtxt("fat.txt")
print(fat_data.shape)

In [ ]:
fat_data

In [ ]:
plot_spectrum(fat_data[200:400, 1], fat_data[200:400, 0])

In [ ]:
water_data = np.loadtxt("water.txt")
print(water_data.shape)

In [ ]:
plot_spectrum(water_data[160:220, 1], water_data[160:220, 0])

In [ ]:
plot_spectra(mu_a_matrix[:, :4].T, loader.wavelengths, ["HbO2", "Hbb", "oxCCO", "redCCO", "Water", "Fat"])

In [ ]:
plot_spectra((*(mu_a_matrix_old[:, -2:].T), *(mu_a_matrix[:, -2:].T + 0.0005)), loader.wavelengths, ["Water - Old", "Fat - Old", "Water - New", "Fat - New"])

In [ ]:
mu_a_matrix_old[:, -2] / mu_a_matrix[:, -2]

In [ ]:
loader.wavelengths

In [ ]:
mcs_obj1 = SimulationAttenuation(config.mcs_func_path.parent / "function_data1.npz")
mcs_obj2 = SimulationAttenuation(config.mcs_func_path.parent / "function_data2.npz")

Create spectrum with MCS and compare to MCS-func

In [ ]:
#g_combined = 0.8675
g_combined = 0.9
refractive_index_combined = 1.38
vol = np.ones((50, 50, 500))
prop = np.array([[0, 0, 1, 1], [0, 0, g_combined, refractive_index_combined]])
unitinmm = 1

In [ ]:
cfg = {
    "nphoton": 5e6, 
    "maxdetphoton": 5e6,
    "unitinmm": unitinmm,
    "vol": vol,
    "tstart":0,
    "tend":1e-8,
    "tstep":1e-8,
    "autopilot": 1,
    "gpuid": config.gpuid,
    "prop":prop,
    "bc": "ccrcca001000", # mark z=0 plane as detector
    "srcdir": [0,0,1],
    "srctype": "planar",
    "srcpos": [0, 0, 0],
    "srcparam1": [vol.shape[0], 0, 0, 0], # 3D position of vertex, fourth coordinate is irrelevant
    "srcparam2": [0, vol.shape[1], 0, 0],
    "issrcfrom0": 1,
    "savedetflag": "dps", # detector id, path length, scatter count
    "flog": config.mcs_func_path / "log.txt",
}

In [ ]:
mu_a_vals = loader.mu_a_func_gray_matter(loader.wavelengths)
mu_s_vals = loader.mu_s_red_func_gray_matter(loader.wavelengths) / (1-g_combined)

In [ ]:
with open(config.result_dir / "A_mcs_generated", "rb") as f:
    A_mcs_generated = pickle.load(f)

In [ ]:
skip_simulation = False

In [ ]:
if not skip_simulation:
    A_mcs_generated = np.empty((loader.wavelengths.shape[0],))

    for i, wl in enumerate(loader.wavelengths):
        cfg["prop"][1, 0] = mu_a_vals[i] / 10
        cfg["prop"][1, 1] = mu_s_vals[i] / 10
        cfg["prop"][1, 2] = g_combined
        cfg["prop"][1, 3] = refractive_index_combined
        print(f"Iteration {i}/{loader.wavelengths.shape[0]}")
        res = pmcx.mcxlab(cfg)
        weights = pmcx.utils.detweight(res["detp"])
        attenuation = -np.log(np.sum(weights)/cfg["nphoton"])
        A_mcs_generated[i] = attenuation

In [ ]:
print(config.result_dir)

In [ ]:
with open(config.result_dir / "A_mcs_generated", "wb") as f:
    pickle.dump(A_mcs_generated, f)

In [ ]:
A_mcs_func_computed1 = mcs_obj1.A_concentrations(
    loader.wavelengths, 
    mu_a_matrix, 
    loader.params_ref_gray_matter[:6],
    *loader.params_ref_gray_matter[-2:]
)

A_mcs_func_computed2 = mcs_obj2.A_concentrations(
    loader.wavelengths, 
    mu_a_matrix, 
    loader.params_ref_gray_matter[:6],
    *loader.params_ref_gray_matter[-2:]
)

In [ ]:
plot_spectra(
    (A_mcs_generated, A_mcs_func_computed1 + 0.03, A_mcs_func_computed2 + 0.02),
    loader.wavelengths,
    labels=["Generated", "Computed1", "Computed2"]
)

In [ ]:
A_mcs_generated / A_mcs_func_computed1[:, 0]

Test fitting with single-thread and without jacobian. See if fitting procedure converges to reference gray matter concentrations, when initialized with slightly perturbed reference concentrations.

In [ ]:
init_vals = loader.params_ref_gray_matter + np.array([0.05, 0.025, 0.001, 0.0005, 0.05, 0.15, 2, 0.5])
print(init_vals)

In [ ]:
concentrations_to_blood_fraction(init_vals)

In [ ]:
loader.params_ref_gray_matter_fraction

In [ ]:
%%time

params_mcs_flat, errors = concentr_fit_nonlinear(
    A_mcs_generated[:, None],
    loader.wavelengths,
    mu_a_matrix,
    mcs_obj1.A_blood_fraction,
    ref_vals=concentrations_to_blood_fraction(init_vals),
    variables_bool_arr=np.array([True, True, True, True, True, True, True, True]),
    left_bounds=np.array([0, 0, 0, 0, 0, 0, 3, 0.1]),
    right_bounds=np.array([1, 1, 0.01, 0.01, 1, 1, 100, 10]),
    is_delta_A=False,
    progress_bar=False,
    verbosity=2
)

In [ ]:
print(params_mcs_flat)

In [ ]:
A_init_reconstr = mcs_obj1.A_concentrations(
    loader.wavelengths,
    mu_a_matrix,
    init_vals[:6],
    *init_vals[-2:]
)

A_reconstr = mcs_obj1.A_blood_fraction(
    loader.wavelengths,
    mu_a_matrix,
    params_mcs_flat[:6, 0],
    *params_mcs_flat[-2:, 0]
)

with open(config.m_params_path, "rb") as f:
    m_params, A_vals, N_vals, dref_vals = pickle.load(f)

A_jacques_reconstr_combinedm = A_jacques_blood_fraction(
    loader.wavelengths,
    mu_a_matrix,
    params_mcs_flat[:6, 0],
    *params_mcs_flat[-2:, 0],
    *m_params[1.38]
)

A_jacques_reconstr_gmm = A_jacques_blood_fraction(
    loader.wavelengths,
    mu_a_matrix,
    params_mcs_flat[:6, 0],
    *params_mcs_flat[-2:, 0],
    *m_params[1.36]
)

In [ ]:
plot_spectra((A_mcs_generated, A_init_reconstr, A_reconstr + 0.03, A_jacques_reconstr_combinedm, A_jacques_reconstr_gmm), loader.wavelengths, ["Generated", "Init", "Reconstructed", "Jacques - combined m", "Jacques - specific m"])

Result: MCS-Func adjusts values so that reconstructed spectrum is essentially identical to input spectrum. Jacques only produces similar spectrum, if m-parameters for correct tissue type are used.

Compute the jacobian using numeric differentiation, and compare to analytic Jacobian.

In [ ]:
eval_vals = loader.params_ref_blood_vessel_fraction
print(eval_vals)

In [ ]:
jac = mcs_obj1.jacobian_blood_fraction(loader.wavelengths, mu_a_matrix, eval_vals[:6], *eval_vals[-2:])

In [ ]:
dx = 1e-7
x = np.tile(eval_vals[:, None], (1, 8))
x_dx = x + dx * np.eye(8)
jac_numeric = mcs_obj1.A_blood_fraction(loader.wavelengths, mu_a_matrix, x_dx[:-2,:], *x_dx[-2:, :]) - mcs_obj1.A_blood_fraction(loader.wavelengths, mu_a_matrix, x[:-2, :], *x[-2:, :])
jac_numeric /= dx

In [ ]:
print(np.isclose(jac[:, 0, :], jac_numeric))

In [ ]:
np.max(np.abs(jac[:, 0, :] - jac_numeric))

Test if using the jacobian works and produces the same results.

In [ ]:
%%time

params_mcs_flat_jacobian, errors = concentr_fit_nonlinear(
    A_mcs_generated[:, None],
    loader.wavelengths,
    mu_a_matrix,
    mcs_obj1.A_blood_fraction,
    jacobian=mcs_obj1.jacobian_blood_fraction,
    ref_vals=concentrations_to_blood_fraction(init_vals),
    variables_bool_arr=np.array([True, True, True, True, True, True, True, True]),
    left_bounds=np.array([0, 0, 0, 0, 0, 0, 3, 0.1]),
    right_bounds=np.array([1, 1, 0.01, 0.01, 1, 1, 100, 10]),
    is_delta_A=False,
    progress_bar=False,
    verbosity=2
)

In [ ]:
A_reconstr_jacobian = mcs_obj1.A_blood_fraction(
    loader.wavelengths,
    mu_a_matrix,
    params_mcs_flat_jacobian[:6, 0],
    *params_mcs_flat_jacobian[-2:, 0]
)

In [ ]:
plot_spectra((A_mcs_generated, A_init_reconstr, A_reconstr + 0.02, A_reconstr_jacobian + 0.03), loader.wavelengths, ["Generated", "Init", "Reconstructed", "Reconstructed with jacobian"])

Using the jacobian works as well, and is about 6 times faster.

Testing for real spectra.

In [ ]:
test_mask = np.zeros_like(loader.label_map, dtype=bool)
test_mask[np.unravel_index(np.random.choice(np.flatnonzero(loader.label_map == 1), size=5, replace=True), loader.label_map.shape)] = True
print(test_mask.shape)

In [ ]:
%time
params_mcs_flat_real, errors = concentr_fit_nonlinear(
    A_gt_img[:, test_mask],
    loader.wavelengths,
    mu_a_matrix,
    mcs_obj1.A_blood_fraction,
    jacobian=mcs_obj1.jacobian_blood_fraction,
    ref_vals=loader.params_ref_gray_matter_fraction,
    variables_bool_arr=np.array([True, True, True, True, True, True, True, True]),
    left_bounds=np.array([0, 0, 0, 0, 0, 0, 3, 0.1]),
    right_bounds=np.array([1, 1, 0.01, 0.01, 1, 1, 100, 10]),
    is_delta_A=False,
    progress_bar=False,
)

In [ ]:
loader.params_ref_gray_matter

In [ ]:
A_reconstr_real = mcs_obj1.A_blood_fraction(loader.wavelengths, mu_a_matrix, params_mcs_flat_real[:6,:], *params_mcs_flat_real[-2:,:])
for i in range(np.count_nonzero(test_mask)):
    plot_spectra((A_gt_img[:, test_mask][:, i], A_reconstr_real[:, i]), loader.wavelengths, labels=["GT", "Reconstructed"])
    print(params_mcs_flat_real[:, i])